In [5]:
!kaggle competitions download -c data-science-bowl-2019


  0%|          | 0.00/399k [00:00<?, ?B/s]
100%|##########| 399k/399k [00:00<00:00, 3.43MB/s]

  0%|          | 0.00/262k [00:00<?, ?B/s]
100%|##########| 262k/262k [00:00<00:00, 8.58MB/s]

  0%|          | 0.00/40.8M [00:00<?, ?B/s]
  7%|7         | 3.00M/40.8M [00:00<00:01, 26.3MB/s]
 12%|#2        | 5.00M/40.8M [00:00<00:01, 21.7MB/s]
 17%|#7        | 7.00M/40.8M [00:00<00:01, 20.7MB/s]
 34%|###4      | 14.0M/40.8M [00:00<00:01, 24.1MB/s]
 51%|#####1    | 21.0M/40.8M [00:00<00:00, 29.8MB/s]
 71%|#######1  | 29.0M/40.8M [00:00<00:00, 36.9MB/s]
 93%|#########3| 38.0M/40.8M [00:00<00:00, 44.9MB/s]
100%|##########| 40.8M/40.8M [00:00<00:00, 47.4MB/s]

  0%|          | 0.00/397M [00:00<?, ?B/s]
  1%|1         | 5.00M/397M [00:00<00:21, 19.0MB/s]
  4%|3         | 15.0M/397M [00:00<00:15, 25.0MB/s]
  6%|6         | 25.0M/397M [00:00<00:12, 32.1MB/s]
  9%|8         | 34.0M/397M [00:00<00:09, 39.9MB/s]
 10%|#         | 41.0M/397M [00:00<00:08, 45.7MB/s]
 12%|#2        | 48.0M/397M [00:00<00

In [10]:
!dir

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 26EF-4FAA

 C:\Users\user\workspace\kaggles\2019-data-science-bowl 디렉터리

11/04/2019  09:41 AM    <DIR>          .
11/04/2019  09:41 AM    <DIR>          ..
11/04/2019  09:34 AM    <DIR>          .ipynb_checkpoints
11/04/2019  09:35 AM             6,857 191104.ipynb
11/04/2019  09:33 AM            11,031 sample_submission.csv
11/04/2019  09:33 AM           408,868 specs.csv
10/21/2019  08:56 PM       398,327,795 test.csv
11/04/2019  09:33 AM        42,791,195 test.csv.zip
10/21/2019  08:56 PM     3,878,373,417 train.csv
11/04/2019  09:33 AM       415,810,137 train.csv.zip
10/21/2019  08:56 PM         1,124,229 train_labels.csv
11/04/2019  09:33 AM           268,195 train_labels.csv.zip
               9개 파일       4,737,121,724 바이트
               3개 디렉터리  96,062,304,256 바이트 남음


# 0. import packages

In [135]:
import pandas as pd
import numpy as np
import json

# 1. read data

In [12]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
label = pd.read_csv('train_labels.csv')
specs = pd.read_csv('specs.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# 2. labels

**there are 5 assessment**
- Bird Measurer
- Cart Balancer
- Cauldron Filler
- Chest Sorter
- Mushroom Sorter

## accuracy_group (label)
- 3: first attempt -> solved
- 2: second attempt -> solved
- 1: 3 or more attempts -> solved
- 0: never solved

set index with `installation_id`  
we should know there are **shared devices issues**

In [ ]:
label.set_index(label['installation_id'], inplace=True)

In [36]:
keys = label.index.unique()

In [53]:
label.loc[[keys[9]]]

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
installation_id,,,,,,,
00634433,1454d273262f61e3,00634433,Cauldron Filler (Assessment),1,0,1.0,3
00634433,440633fe3ef55b90,00634433,Cart Balancer (Assessment),1,0,1.0,3
00634433,8c2d5dff4635d020,00634433,Chest Sorter (Assessment),0,1,0.0,0


In [67]:
train.head(10)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK
5,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.279Z,"{""coordinates"":{""x"":583,""y"":605,""stage_width"":...",0001e90f,4,4070,9991,Sandcastle Builder (Activity),Activity,MAGMAPEAK
6,1325467d,0848ef14a8dc6892,2019-09-06T17:55:06.913Z,"{""coordinates"":{""x"":601,""y"":570,""stage_width"":...",0001e90f,5,4070,10622,Sandcastle Builder (Activity),Activity,MAGMAPEAK
7,1325467d,0848ef14a8dc6892,2019-09-06T17:55:07.546Z,"{""coordinates"":{""x"":250,""y"":665,""stage_width"":...",0001e90f,6,4070,11255,Sandcastle Builder (Activity),Activity,MAGMAPEAK
8,1325467d,0848ef14a8dc6892,2019-09-06T17:55:07.979Z,"{""coordinates"":{""x"":279,""y"":629,""stage_width"":...",0001e90f,7,4070,11689,Sandcastle Builder (Activity),Activity,MAGMAPEAK
9,1325467d,0848ef14a8dc6892,2019-09-06T17:55:08.566Z,"{""coordinates"":{""x"":839,""y"":654,""stage_width"":...",0001e90f,8,4070,12272,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [58]:
test.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10T16:50:24.910Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10T16:50:55.503Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10T16:51:51.805Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10T16:53:12.825Z,"{""event_code"": 2000, ""event_count"": 1}",00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10T16:54:12.115Z,"{""version"":""1.0"",""round"":0,""event_count"":1,""ga...",00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [59]:
specs.head()

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [168]:
specs[specs['event_id']=='27253bdc']['info'].iloc[0]

'This event is triggered when a video starts playing'

In [160]:
a = json.loads(specs['args'][1])

In [164]:
specs['args2'] = specs['args'].transform(json.loads)

In [167]:
for i in specs['args2']:
    print(len(i))

9
7
8
8
8
7
9
8
8
9
6
5
8
9
8
7
7
6
7
8
5
5
7
8
7
8
5
3
8
8
7
11
5
9
5
12
10
5
8
9
7
4
6
7
6
7
10
7
7
8
7
6
8
8
8
8
6
6
8
8
4
7
8
12
8
4
8
9
8
7
7
8
9
5
5
8
5
4
6
4
7
7
8
9
7
10
7
7
6
8
8
7
7
5
4
7
8
9
7
6
5
5
8
5
6
5
5
8
4
8
8
5
5
8
5
6
5
7
8
7
12
7
7
8
7
9
3
8
4
8
5
7
7
5
7
8
6
4
7
6
7
6
11
5
7
5
6
8
8
7
4
7
5
4
4
5
8
5
8
4
5
5
7
7
4
6
7
8
4
9
6
8
4
5
7
4
4
8
7
5
5
7
4
8
9
5
8
6
5
7
5
8
4
6
3
5
5
5
5
8
5
6
5
5
5
8
8
8
10
8
8
10
5
4
7
7
8
6
10
6
8
7
12
6
5
8
9
7
6
7
4
7
8
5
6
8
10
10
8
6
6
4
8
5
10
8
4
4
4
4
8
8
4
11
6
11
5
5
6
7
8
8
8
4
5
9
8
5
9
7
6
7
7
7
5
4
8
7
4
7
7
11
7
6
8
7
6
7
7
6
8
7
9
5
9
7
4
7
5
6
8
6
4
7
7
6
6
7
8
5
8
5
8
6
8
6
5
8
3
5
5
9
7
6
9
8
8
6
11
5
5
8
5
7
7
6
5
8
6
7
8
4
4
2
6
6
8
6
7
4
9
4
4
6
4
5
6
8
8
8
5
6
5
6
4
4
7
6
5
4
6
6
5
5
4
5
4
5
7
5
6
4
4
7
4
6
